# Create an empty Ubuntu VM on Azure

Install HAGrid

```
pip install hagrid
```

__NOTE__: Just in case there are some issues with a VM we should provision are few extra, here we use node_count 12 for a session of 10 users

Run hagrid launch with these arguments:

```
hagrid launch to azure --image_name=domain_0.7.0 --jupyter --ansible_extras="install=false" --node_count 12
```

- Use a new unique resource group for this session like: aa-test-1
- Choose the location where your demo participants will be located, e.g. `eastus`, `westus` etc
- Choose an 8 core machine like `Standard_D8s_v3`
- Set the username to `azureuser`
- Choose password and then `n` to auto-generate password
- Set an easy to remember 12 character password like: `Adastrademo2022`
- What ever you enter into Repo and Branch will be ignored

![ip_address](img/hagrid_bare_vm.png)

After it is finished you should see this message

![ip_address](img/hagrid_bare_vm_output.png)

Now run this to get JSON containing the information for all the VMs:

```
cat ~/.hagrid/host_ips.json
```

In [1]:
import os
import json

# paste the path to host ip json here
HOST_IP_PATH = "~/.hagrid/host_ips.json"
HOST_IP_PATH = os.path.expanduser(HOST_IP_PATH)

with open(HOST_IP_PATH) as fp:
    host_ips = json.loads(fp.read())

In [2]:
host_ips

{'host_ips': [{'username': 'azureuser',
   'password': 'Adastrademo2022',
   'ip_address': '20.85.158.49',
   'jupyter_token': '3gq5h89241qyfnncj6q6dxhq9ut9ttmrofm49e3ghiqoggdn'},
  {'username': 'azureuser',
   'password': 'Adastrademo2022',
   'ip_address': '20.85.155.54',
   'jupyter_token': 'c67xn8chbdw1t3a2l27baqe8sic2rsvua6brlzam7y9ogrcm'},
  {'username': 'azureuser',
   'password': 'Adastrademo2022',
   'ip_address': '20.85.157.249',
   'jupyter_token': 't9wvjucg36q9u7lnc4gx1nhpxhbizmvu604zkowahk3zk5gu'},
  {'username': 'azureuser',
   'password': 'Adastrademo2022',
   'ip_address': '20.85.159.232',
   'jupyter_token': 'y7ybcicbat4eyosbdna1t1k4y3x0e79hyfed8upgkxulhagy'},
  {'username': 'azureuser',
   'password': 'Adastrademo2022',
   'ip_address': '20.85.159.130',
   'jupyter_token': 'f7wyep2sjb5mt7nogufuomx6yj254i1rlmeu0xh31i9nzte4'},
  {'username': 'azureuser',
   'password': 'Adastrademo2022',
   'ip_address': '20.85.158.23',
   'jupyter_token': 'a8al302udmrffcwdc4i6h3e7u373f

In [3]:
# update TOTAL_PARTICIPANTS
# use the total participants not the total machines, e.g. 10 not 12
# as this is used to calculate the data split assignment
TOTAL_PARTICIPANTS = 10

In [4]:
# optionally add names or emails here which will be printed below to help keep track of assignment
participants = [
    "Teo",
    "Ruchi",
    "Kyoko",
    "Ivy",
    "Shubham",
    "Irina",
    "Laura",
    "Ionesio",
    "Ronnie",
    "Rasswanth"
]
print("Total participants:", len(participants))

Total participants: 10


In [5]:
def check_ip_port(host_ip: str, port: int) -> bool:
    import socket
    try:
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.settimeout(2)
        result = sock.connect_ex((host_ip, port))
        sock.close()
        if result == 0:
            return True
    except Exception:
        pass
    return False

In [6]:
def get_icon(status: bool) -> str:
    return "✅" if status else "❌"

In [7]:
def check_hosts_ready(host_ips: dict) -> None:
    for host in host_ips["host_ips"]:
        print("-----------------------")
        host_ip = host["ip_address"]
        # make sure the containers are not running
        http_up = check_ip_port(host_ip=host_ip, port=80)
        print(f"{get_icon(not http_up)} Containers Off {host_ip}:80")
        
        # make sure jupyter notebooks is up
        jupyter_up = check_ip_port(host_ip=host_ip, port=8888)
        print(f"{get_icon(jupyter_up)} Jupyter Up {host_ip}:8888")
        
        # make sure SSH is up
        ssh_up = check_ip_port(host_ip=host_ip, port=22)
        print(f"{get_icon(ssh_up)} SSH Up {host_ip}:22")

        print()
        all_status = (not http_up) and jupyter_up and ssh_up
        print(f"{get_icon(all_status)} Node {host_ip} Ready!")
        print("-----------------------")
        print()

In [8]:
check_hosts_ready(host_ips)

-----------------------
✅ Containers Off 20.85.158.49:80
✅ Jupyter Up 20.85.158.49:8888
✅ SSH Up 20.85.158.49:22

✅ Node 20.85.158.49 Ready!
-----------------------

-----------------------
✅ Containers Off 20.85.155.54:80
✅ Jupyter Up 20.85.155.54:8888
✅ SSH Up 20.85.155.54:22

✅ Node 20.85.155.54 Ready!
-----------------------

-----------------------
✅ Containers Off 20.85.157.249:80
✅ Jupyter Up 20.85.157.249:8888
✅ SSH Up 20.85.157.249:22

✅ Node 20.85.157.249 Ready!
-----------------------

-----------------------
✅ Containers Off 20.85.159.232:80
✅ Jupyter Up 20.85.159.232:8888
✅ SSH Up 20.85.159.232:22

✅ Node 20.85.159.232 Ready!
-----------------------

-----------------------
✅ Containers Off 20.85.159.130:80
✅ Jupyter Up 20.85.159.130:8888
✅ SSH Up 20.85.159.130:22

✅ Node 20.85.159.130 Ready!
-----------------------

-----------------------
✅ Containers Off 20.85.158.23:80
✅ Jupyter Up 20.85.158.23:8888
✅ SSH Up 20.85.158.23:22

✅ Node 20.85.158.23 Ready!
-----------------

In [9]:
def output_user_details(host_ips: dict, participants: list[str] = []) -> None:
    notebook_path = "adastra/data-owners/00-data-owners-create-domain.ipynb"
    print("===============================")
    print("Ad Astra 🚀🌌")
    print("===============================")
    print()
    print("Send to each participant")
    print()
    if TOTAL_PARTICIPANTS > len(host_ips["host_ips"]):
        raise Exception(f"TOTAL_PARTICIPANTS: {TOTAL_PARTICIPANTS} is less than VM count: {len(host_ips['host_ips'])}")
    partition = 0
    for host in host_ips["host_ips"]:
        partition += 1
        if partition <= len(participants):
            print(f"Hi {participants[partition - 1]},")
        if partition <= TOTAL_PARTICIPANTS:
            print("These are your Session Details:")
        else:
            print("Spare Session Details:")
        print("-------------------------------")
        print(f"Username: {host['username']}")
        print(f"Password: {host['password']}")
        print(f"VM IP Address: {host['ip_address']}")
        if partition <= TOTAL_PARTICIPANTS:
            print()
            print(f"MY_PARTICIPANT_NUMBER={partition}")
            print(f"TOTAL_PARTICIPANTS={TOTAL_PARTICIPANTS}")

        print()
        print(f"👉🏽 Start Here:")
        print(
            f"http://{host['ip_address']}:8888/lab/tree/notebooks/{notebook_path}"
            f"?token={host['jupyter_token']}"
        )
        print()

In [10]:
output_user_details(host_ips, participants)

Ad Astra 🚀🌌

Send to each participant

Hi Teo,
These are your Session Details:
-------------------------------
Username: azureuser
Password: Adastrademo2022
VM IP Address: 20.85.158.49

MY_PARTICIPANT_NUMBER=1
TOTAL_PARTICIPANTS=10

👉🏽 Start Here:
http://20.85.158.49:8888/lab/tree/notebooks/adastra/data-owners/00-data-owners-create-domain.ipynb?token=3gq5h89241qyfnncj6q6dxhq9ut9ttmrofm49e3ghiqoggdn

Hi Ruchi,
These are your Session Details:
-------------------------------
Username: azureuser
Password: Adastrademo2022
VM IP Address: 20.85.155.54

MY_PARTICIPANT_NUMBER=2
TOTAL_PARTICIPANTS=10

👉🏽 Start Here:
http://20.85.155.54:8888/lab/tree/notebooks/adastra/data-owners/00-data-owners-create-domain.ipynb?token=c67xn8chbdw1t3a2l27baqe8sic2rsvua6brlzam7y9ogrcm

Hi Kyoko,
These are your Session Details:
-------------------------------
Username: azureuser
Password: Adastrademo2022
VM IP Address: 20.85.157.249

MY_PARTICIPANT_NUMBER=3
TOTAL_PARTICIPANTS=10

👉🏽 Start Here:
http://20.85.157.24